<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test

In [ ]:
using System;
using System.Collections.Generic;
using System.ComponentModel;
using System.Data;
using System.Diagnostics;
using System.Drawing;
using System.Linq;
using System.Runtime.InteropServices;
using System.Text;
using System.Threading;
using System.Threading.Tasks;
using System.Windows.Forms;
using static System.Windows.Forms.VisualStyles.VisualStyleElement;
// CircularProgressBar
namespace WindowsFormsApp2
{
    public partial class Form1 : Form
    {
        public Form1()
        {
            InitializeComponent();
            HookManager.MouseMove += HookManager_MouseMove;
            Application.ApplicationExit += Application_ApplicationExit;
            HookManager.Init();
        }

        private void Form1_Load(object sender, EventArgs e)
        {
            _current_pos = System.Windows.Forms.Cursor.Position;
            this.Visible = false;
        }
        int _count = 0;
        Point _current_pos = new Point();
        bool _is_same_pos = false;
        private void timer1_Tick(object sender, EventArgs e)
        {
            //circleprogressBar1.updateProgress(_count++);
            if (_current_pos == System.Windows.Forms.Cursor.Position)
            {
                circularProgressBar1.Value = _count++;
                this.Location = new Point(Cursor.Position.X - this.Width / 2, Cursor.Position.Y - this.Height / 2);
                this.Visible = true;
                this.TopMost = true;
            }

            if (_count == 4)
            {
                // stuff here
                Thread.Sleep(1000);
                _count = 0;
                this.Visible = false;
            }
        }

        private void Application_ApplicationExit(object sender, EventArgs e)
        {
            // Gỡ bỏ Hook khi ứng dụng thoát
            HookManager.MouseMove -= HookManager_MouseMove;
        }

        private void HookManager_MouseMove(object sender, MouseEventArgs e)
        {
            // Xử lý sự kiện chuột di chuyển ở đây
            _current_pos = new Point(e.X, e.Y);
            this.Visible = false;
            _count = 0;
        }
    }

    public static class HookManager
    {
        // Hằng số cho sự kiện chuột di chuyển (WM_MOUSEMOVE)
        private const int WH_MOUSE_LL = 14;
        private const int WM_MOUSEMOVE = 0x0200;

        // Khai báo các hàm API từ thư viện user32.dll
        private delegate IntPtr LowLevelMouseProc(int nCode, IntPtr wParam, IntPtr lParam);
        private static LowLevelMouseProc _proc = HookCallback;
        private static IntPtr _hookID = IntPtr.Zero;

        public static event MouseEventHandler MouseMove;

        // Khởi tạo và gắn Hook
        public static void Init()
        {
            _hookID = SetHook(_proc);
        }

        // Gỡ bỏ Hook
        public static void Unhook()
        {
            UnhookWindowsHookEx(_hookID);
        }

        // Thiết lập Hook
        private static IntPtr SetHook(LowLevelMouseProc proc)
        {
            using (Process curProcess = Process.GetCurrentProcess())
            using (ProcessModule curModule = curProcess.MainModule)
            {
                return SetWindowsHookEx(WH_MOUSE_LL, proc, GetModuleHandle(curModule.ModuleName), 0);
            }
        }

        // Hàm xử lý sự kiện Hook
        private static IntPtr HookCallback(int nCode, IntPtr wParam, IntPtr lParam)
        {
            if (nCode >= 0 && wParam == (IntPtr)WM_MOUSEMOVE)
            {
                MSLLHOOKSTRUCT hookStruct = (MSLLHOOKSTRUCT)Marshal.PtrToStructure(lParam, typeof(MSLLHOOKSTRUCT));
                MouseMove?.Invoke(null, new MouseEventArgs(MouseButtons.None, 0, hookStruct.pt.x, hookStruct.pt.y, 0));
            }
            return CallNextHookEx(_hookID, nCode, wParam, lParam);
        }

        // Cấu trúc chứa thông tin về sự kiện chuột
        [StructLayout(LayoutKind.Sequential)]
        private struct MSLLHOOKSTRUCT
        {
            public POINT pt;
            public uint mouseData;
            public uint flags;
            public uint time;
            public IntPtr dwExtraInfo;
        }

        // Cấu trúc POINT dùng để lưu tọa độ chuột
        [StructLayout(LayoutKind.Sequential)]
        private struct POINT
        {
            public int x;
            public int y;
        }

        // Import các hàm API từ user32.dll
        [DllImport("user32.dll", CharSet = CharSet.Auto, SetLastError = true)]
        private static extern IntPtr SetWindowsHookEx(int idHook, LowLevelMouseProc lpfn, IntPtr hMod, uint dwThreadId);

        [DllImport("user32.dll", CharSet = CharSet.Auto, SetLastError = true)]
        [return: MarshalAs(UnmanagedType.Bool)]
        private static extern bool UnhookWindowsHookEx(IntPtr hhk);

        [DllImport("user32.dll", CharSet = CharSet.Auto, SetLastError = true)]
        private static extern IntPtr CallNextHookEx(IntPtr hhk, int nCode, IntPtr wParam, IntPtr lParam);

        [DllImport("kernel32.dll", CharSet = CharSet.Auto, SetLastError = true)]
        private static extern IntPtr GetModuleHandle(string lpModuleName);

    }
}


In [ ]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Runtime.Serialization.Formatters.Binary;
using System.Runtime.Serialization;
using System.Security.Cryptography;
using System.Text;
using System.Threading.Tasks;

namespace WindowsFormsApp2
{
    public class A_Serialization
    {

        // if (!A_Serialization.Main.Serializing(_file_info.FullName, System_Data))
        // System_Data = (System_Data)A_Serialization.Main.Deserializing(_file_info.FullName);
        public static A_Serialization Main;
        private string _error_message = "";
        public string Error_Message { get { return _error_message; } }
        byte[] key = { 1, 2, 3, 4, 5, 6, 7, 8 }; // Where to store these keys is the tricky part,
        byte[] iv = { 1, 2, 3, 4, 5, 6, 7, 8 };

        void CreateFilePath(string FilePath)
        {
            if (!File.Exists(FilePath))
            {
                // File Path에서 Directory Path를 받아옮
                string dc = Path.GetDirectoryName(FilePath);

                // Directory가 없을 경우 생성
                if (!Directory.Exists(dc))
                    Directory.CreateDirectory(dc);

                using (File.Create(FilePath)) { };
            }
        }

        /// <summary>
        /// Object를 저장파일로 Serializing
        /// </summary>
        /// <param name="Path"></param>
        /// <param name="Obj"></param>
        /// <returns></returns>
        public bool Serializing(string Path, Object Obj)
        {
            Stream memoryStream = new MemoryStream();

            try
            {
                CreateFilePath(Path);

                // Binary 형태로 serialzie 하기 위한 포맷 생성
                BinaryFormatter formatter = new BinaryFormatter();

                // obj 개체를 스트림 serialize
                formatter.Serialize(memoryStream, Obj);

                // 파일을 쓰기 위해 스트림 생성
                // using 사용시에 구문 끝나면 자동 스트림 삭제
                using (Stream fileStream = new FileStream(Path, FileMode.Create, FileAccess.Write, FileShare.None))
                {
                    memoryStream.Position = 0;
                    memoryStream.CopyTo(fileStream);
                    fileStream.Dispose();
                }

                return true;
            }
            catch (Exception ex)
            {
                _error_message = "[Serializing] " + ex.ToString();
                return false;
            }
            finally
            {
                if (memoryStream != null) memoryStream.Dispose();
            }
        }

        /// <summary>
        /// 저장파일을 Object로 Deserializing
        /// </summary>
        /// <param name="Path"></param>
        /// <returns></returns>
        public Object Deserializing(string Path)
        {
            if (!File.Exists(Path))
            {
                _error_message = string.Format("System file is not existed! \t{0}", Path);
                return null;
            }

            // 새로운 Object 생성
            Object Obj = new Object();

            // Binary 형태로 serialzie 하기 위한 포맷 생성
            BinaryFormatter formatter = new BinaryFormatter();

            try
            {
                // 파일을 읽기 위해 스트림 생성
                // using 사용시에 구문 끝나면 자동 스트림 삭제
                using (Stream stream = new FileStream(Path, FileMode.Open, FileAccess.Read, FileShare.Read))
                {
                    // 스트림을 Deserialize하여 Obj개체 생성
                    Obj = (Object)formatter.Deserialize(stream);
                    stream.Dispose();
                }

                return Obj;
            }
            catch (Exception ex)
            {
                _error_message = "[Deserializing] " + ex.ToString();
                return null;
            }
        }

        /// <summary>
        /// No Use
        /// </summary>
        /// <param name="_input_object"></param>
        /// <param name="_path"></param>
        /// <returns></returns>
        public bool Serialize(object _input_object, string _path)
        {
            try
            {
                DESCryptoServiceProvider des = new DESCryptoServiceProvider();
                using (MemoryStream _ms = Serialize_To_Stream(_input_object))
                {
                    using (var cryptoStream = new CryptoStream(_ms, des.CreateEncryptor(key, iv), CryptoStreamMode.Write))
                    {

                        //using (StreamWriter _sw = new StreamWriter(cryptoStream))
                        //{
                        //    _sw.Write("HiHIHIHIHI");
                        //    _sw.Close();
                        //}

                        cryptoStream.Close();
                    }
                    _ms.Close();
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        /// <summary>
        /// No Use
        /// </summary>
        /// <param name="_input_object"></param>
        /// <param name="_path"></param>
        /// <returns></returns>
        public object Deserialize(string _path)
        {
            try
            {
                object _object;
                DESCryptoServiceProvider des = new DESCryptoServiceProvider();
                using (Stream _stream_reader = new FileStream(_path, FileMode.OpenOrCreate, FileAccess.ReadWrite, FileShare.ReadWrite))
                {
                    using (var cryptoStream = new CryptoStream(_stream_reader, des.CreateDecryptor(key, iv), CryptoStreamMode.Read))
                    {
                        //using (StreamReader _sr = new StreamReader(cryptoStream))
                        //{
                        BinaryFormatter formatter = new BinaryFormatter();
                        cryptoStream.Seek(0, SeekOrigin.Begin);
                        _object = formatter.Deserialize(cryptoStream);
                        //cryptoStream.Close();
                        //}
                    }

                    _stream_reader.Close();
                }
                return null;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return null;
            }
        }

        private static MemoryStream Serialize_To_Stream(object o)
        {
            MemoryStream stream = new MemoryStream();
            IFormatter formatter = new BinaryFormatter();
            formatter.Serialize(stream, o);
            return stream;
        }

        private static object Deserialize_From_Stream(MemoryStream stream)
        {
            IFormatter formatter = new BinaryFormatter();
            stream.Seek(0, SeekOrigin.Begin);
            object o = formatter.Deserialize(stream);
            return o;
        }
    }

    public static class EnumHelper
    {
        /// <summary>
        /// Gets an attribute on an enum field value
        /// </summary>
        /// <typeparam name="T">The type of the attribute you want to retrieve</typeparam>
        /// <param name="enumVal">The enum value</param>
        /// <returns>The attribute of type T that exists on the enum value</returns>
        /// <example><![CDATA[string desc = myEnumVariable.GetAttributeOfType<DescriptionAttribute>().Description;]]></example>
        public static T GetAttributeOfType<T>(this Enum enumVal) where T : System.Attribute
        {
            var type = enumVal.GetType();
            var memInfo = type.GetMember(enumVal.ToString());
            var attributes = memInfo[0].GetCustomAttributes(typeof(T), false);
            return (attributes.Length > 0) ? (T)attributes[0] : null;
        }
    }
}


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace WindowsFormsApp2
{
    public static class Global_Configuration
    {
        /// <summary>
        /// Đường dẫn Model.
        /// </summary>
        public static string Audio_Model_Path { get; set; }
        /// <summary>
        /// Index Audio Hw.
        /// </summary>
        public static int Audio_Hw_Index { get; set; }
        /// <summary>
        /// Đường dẫn file Model phát hiện khuôn mặt.
        /// </summary>
        public static string Face_Landmakrs_Model_Path { get; set; }
        /// <summary>
        /// Index Camera Hw.
        /// </summary>
        public static int Camera_Hw_Index { get; set; }
        public static int Camera_Exposure_Time { get; set; }

    }
}
